## [Working Title]

Created by Toomas Erik Anijärv in 25.05.2023

This notebook is a representation of EEG processing done for the publication with one of the participants as an example.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

In [ ]:
# Import packages
import mne, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from autoreject import (get_rejection_threshold, AutoReject)
from fooof import FOOOF
from fooof.plts.spectra import plot_spectra, plot_spectra_shading

# Set the default directory
os.chdir('/Users/aliciacampbell/Documents/GitHub/EEG-pyline')
mne.set_log_level('error')

# Import functions
import basic.arrange_data as arrange
import signal_processing.pre_process as prep
import signal_processing.spectral_analysis as spectr
import signal_processing.erp_analysis as erpan

In [ ]:
# Folder where to get the raw EEG files
raw_folder = 'Data/Raw/'

# Folder where to export the clean epochs files
clean_folder = 'Data/Clean/'

# Folder where to save the results and plots
results_folder = 'Results/'

# Sub-folder for the experiment (i.e. timepoint or group)
exp_folder = 'LEISURE/T1/SART/'
exp_condition = 'SART_T1'

### PRE-PROCESSING & TASK PERFORMANCE

In [ ]:
# EOG + mastoid channels and stimulus channel
eog_channels = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
stimulus_channel = 'Status'

# Parameters for filter design
filter_design = dict(l_freq=1, h_freq=30, filter_length='auto', method='fir',
                     l_trans_bandwidth='auto', h_trans_bandwidth='auto',
                     phase='zero', fir_window='hamming', fir_design='firwin')

# Epoch time window from event/stimuli
tminmax = [-0.5, 0.8]

# Baseline correction time window
baseline_correction = None

# Event names with IDs for GO and NO-GO trials
event_dict = {'GO trial': 4,
              'NO-GO trial': 8,
              'Button press': 16}

# Button press ID
button_id = 16

In [ ]:
# Get directories of raw EEG files and set export directory for clean files
dir_inprogress = os.path.join(raw_folder,exp_folder)
export_dir = os.path.join(clean_folder,exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress,'.bdf')

In [ ]:
# Loop through all the subjects' directories (EEG files directories)
df_success = pd.DataFrame()
for i in range(len(file_dirs)):
        print('\n{} ({} / {})'.format(subject_names[i], i+1, len(file_dirs)))
        # Read in the raw EEG data
        raw = mne.io.read_raw_bdf(file_dirs[i], infer_types=True, eog=eog_channels,
                                  stim_channel=stimulus_channel)
        try:
                raw = raw.drop_channels(["Erg1",
                              "B1","B2","B3","B4","B5","B6","B7","B8","B9","B10","B11","B12",
                              "B13","B14","B15","B16","B17","B18","B19","B20","B21","B22","B23","B24",
                              "B25","B26","B27","B28","B29","B30","B31","B32",
                              "C1","C2","C3-1","C4-1","C5","C6","C7","C8","C9","C10","C11","C12",
                              "C13","C14","C15","C16","C17","C18","C19","C20","C21","C22","C23","C24",
                              "C25","C26","C27","C28","C29","C30","C31","C32",
                              "D1","D2","D3","D4","D5","D6","D7","D8","D9","D10","D11","D12",
                              "D13","D14","D15","D16","D17","D18","D19","D20","D21","D22","D23","D24",
                              "D25","D26","D27","D28","D29","D30","D31","D32"]).rename_channels({'C3-0':'C3','C4-0':'C4'})
                print('Wrong montage. Dropping many electrodes and renaming some to match the right one.')
        except:
                try:
                        raw = raw.drop_channels(["Erg1"])
                        print('Correct montage, but dropping respiratory data.')
                except:
                        raw = raw
                        print('Correct montage.')

        # Set the right montage (Biosemi32) and set reference as average across all channels
        raw = raw.set_montage(mne.channels.make_standard_montage('biosemi32')).load_data()\
                 .set_eeg_reference(ref_channels='average', verbose=False)
    
        # Filter the signal with bandpass filter and remove EOG artefacts with SSP
        filt = prep.filter_raw_data(raw, filter_design, line_remove=None, eog_channels=eog_channels,
                                    plot_filt=False, savefig=False, verbose=False)
        
        # Find events from the filtered EEG data and name them
        events = mne.find_events(filt, stim_channel=stimulus_channel, consecutive=False, output='onset')
        
        # Create an array and dataframe of successful GO (followed with button press) and NO-GO trials (no button press)
        success_events_total = []
        success_go = []
        unsuccess_nogo = []
        df_success_temp = pd.DataFrame(index=[subject_names[i]],
                                       data={'Total GO' : np.sum(events[:, 2] == event_dict['GO trial']),
                                             'Total NO-GO' : np.sum(events[:, 2] == event_dict['NO-GO trial']),
                                             'Correct GO': 0, 'Correct NO-GO': 0,
                                             'Incorrect GO': 0, 'Incorrect NO-GO': 0})
        # Go through all events to check if they were successful or not
        for m in range(len(events)):
                # If event is a GO, check for button press
                if events[m][2] == event_dict['GO trial']:
                        if events[m+1][2] == button_id:
                                # If there is a button press -> success
                                success_events_total.append(events[m])
                                success_events_total.append(events[m+1])
                                success_go.append(events[m])
                                success_go.append(events[m+1])
                                df_success_temp['Correct GO'] += 1
                        else:
                                # If there is no button press -> fail
                                df_success_temp['Incorrect GO'] += 1
                # If event is a NO-GO, check for no button press
                if events[m][2] == event_dict['NO-GO trial']:
                        if events[m+1][2] != button_id:
                                # If there is no button press -> success
                                success_events_total.append(events[m])
                                df_success_temp['Correct NO-GO'] += 1
                        else:
                                # If there is a button press -> fail
                                unsuccess_nogo.append(events[m])
                                unsuccess_nogo.append(events[m+1])
                                df_success_temp['Incorrect NO-GO'] += 1
        success_events_total = np.asarray(success_events_total)
        success_go = np.asarray(success_go)
        unsuccess_nogo = np.asarray(unsuccess_nogo)

        # Calculate response times to button press for correct GO and incorrect NO-GO
        if len(success_go)!=0:
                rt_go = np.diff(success_go[:, 0])[0::2]/raw.info['sfreq']
        else:
                rt_go = 0
        if len(unsuccess_nogo)!=0:
                rt_nogo = np.diff(unsuccess_nogo[:, 0])[0::2]/raw.info['sfreq']
        else:
                rt_nogo = 0

        # Calculate descriptives for these response times
        df_success_temp['Average RT (Correct GO)'] = np.mean(rt_go)
        df_success_temp['Average RT (Incorrect NO-GO)'] = np.mean(rt_nogo)
        df_success_temp['SD RT (Correct GO)'] = np.std(rt_go)
        df_success_temp['SD RT (Incorrect NO-GO)'] = np.std(rt_nogo)
        df_success_temp['Median RT (Correct GO)'] = np.median(rt_go)
        df_success_temp['Median RT (Incorrect NO-GO)'] = np.median(rt_nogo)
        df_success_temp['Minimum RT (Correct GO)'] = np.min(rt_go)
        df_success_temp['Minimum RT (Incorrect NO-GO)'] = np.min(rt_nogo)
        df_success_temp['Maximum RT (Correct GO)'] = np.max(rt_go)
        df_success_temp['Maximum RT (Incorrect NO-GO)'] = np.max(rt_nogo)
        df_success_temp['RTs (Correct GO)'] = str(rt_go)
        df_success_temp['RTs (Incorrect NO-GO)'] = str(rt_nogo)

        # Merge the participant dataframe with the master dataframe
        df_success = pd.concat([df_success, df_success_temp])

        # Plot all the events
        %matplotlib inline
        fig, axs = plt.subplots(1, 1, figsize=(10, 5))
        fig = mne.viz.plot_events(success_events_total, sfreq=filt.info['sfreq'],
                                  first_samp=filt.first_samp, event_id=event_dict,
                                  axes=axs, show=False)
        fig.subplots_adjust(right=0.7)
        axs.set_title('Successful events ({})'.format(subject_names[i]))
        plt.show()

        # Create epochs time-locked to successful GO and NO-GO events (without including the button press events)
        picks = mne.pick_types(filt.info, eeg=True, stim=False)
        epochs = mne.Epochs(filt, success_events_total[success_events_total[:, 2] != 16], #event_id=event_dict,
                            tmin=tminmax[0], tmax=tminmax[1], baseline=baseline_correction,
                            picks=picks, preload=True)
        
        # Plot the epochs' GFP plot before artefact rejection
        epochs.plot_image(title="GFP without AR ({})".format(subject_names[i]))

        # Use AutoReject to repair and remove epochs which are artefactual
        reject_criteria = get_rejection_threshold(epochs)
        print('Dropping epochs with rejection threshold:',reject_criteria)
        epochs.drop_bad(reject=reject_criteria)

        ar = AutoReject(thresh_method='random_search', random_state=1)
        ar.fit(epochs)
        epochs_ar, reject_log = ar.transform(epochs, return_log=True)
        reject_log.plot('horizontal')

        # Plot the epochs' GFP after artefact rejection
        epochs_ar.average().plot()
        epochs_ar.plot_image(title="GFP with AR ({})".format(subject_names[i]))

        # Display the final epochs object meta-data
        display(epochs_ar)

        # Save the cleaned EEG file as .fif file
        try:
                os.makedirs(export_dir)
        except FileExistsError:
                pass
        try:
                mne.Epochs.save(epochs_ar, fname='{}/{}_clean-epo.fif'.format(export_dir,
                                                                              subject_names[i]),
                                                                              overwrite=True)
        except FileExistsError:
                pass
        
# Save the dataframe for task performance
df_success.to_excel('{}/{}/{}_task_performance.xlsx'.format(results_folder,exp_folder,exp_condition))

### SPECTRAL ANALYSIS: APERIODIC + THETA ACTIVITY

In [ ]:
# Brain regions and their channels /// do for Fz, Cz, Pz
# region = {'Global' : ['Fp1', 'AF3', 'F3', 'FC1', 'Fp2', 'AF4', 'F4', 'FC2', 'Fz',
#                             'F7', 'FC5', 'T7', 'C3', 'F8', 'FC6', 'T8', 'C4', 'Cz',
#                             'CP5', 'P3', 'P7', 'CP6', 'P4', 'P8', 'CP1', 'CP2', 'Pz',
#                             'PO3', 'PO4', 'O1', 'O2', 'Oz']}

region = {'Parieto-occipital' : ['P3', 'P7', 'P4', 'P8', 'Pz',
                                 'PO3', 'PO4', 'O1', 'O2', 'Oz']}

# region = {'Fronto-central' : ['F3', 'FC1', 'F4', 'FC2', 'Fz', 'C3', 'C4', 'Cz', 'CP5', 'CP6', 'CP1', 'CP2']}

# region = {'Central' : ['C3', 'C4', 'Cz', 'CP5', 'CP6', 'CP1', 'CP2']}
# region = {'Midline' : ['Fz', 'Cz', 'Pz']}

# Power spectra estimation parameters
psd_params = {'pre' : dict(method='welch', fminmax=[1, 30], window='hamming', window_duration=0.5,
                           window_overlap=0, zero_padding=3, tminmax=[-0.5, 0]),
              'post' : dict(method='welch', fminmax=[1, 30], window='hamming', window_duration=0.8,
                            window_overlap=0, zero_padding=3, tminmax=[0, 0.8])}

# FOOOF (specparam) model parameters
fooof_params = dict(peak_width_limits=[1, 12], max_n_peaks=float('inf'), min_peak_height=0.225,
                    peak_threshold=2.0, aperiodic_mode='fixed')

# Band power of interest
bands = {'Theta' : [4, 8]}

# Flattened spectra amplitude scale (linear, log)
flat_spectr_scale = 'linear'

# Plot more information on the model fit plots or not; and save these plots or not
plot_rich = True
savefig = True

# Event names (i.e. different stimuli) within the epochs
# event_list = ['GO trial', 'NO-GO trial']
event_list = {'4' : 'GO trial',
              '8' : 'NO-GO trial'}

In [ ]:
# Get directories of clean EEG files and set export directory
dir_inprogress = os.path.join(clean_folder, exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress, "_clean-epo.fif")

In [ ]:
# Pre-create results folders and dataframe
arrange.create_results_folders(exp_folder=exp_folder, results_folder=results_folder, fooof=True)
df_ch = pd.DataFrame()
# Go through all the files (subjects) in the folder
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]),
                                                                verbose=False)

    # Loop through all different events
    df_ch_ev_pre = pd.DataFrame()
    df_ch_ev_post = pd.DataFrame()
    df_ch_ev_post_diff = pd.DataFrame()
    for ev, evname in event_list.items():
        print('{} for {} ({}/{})'.format(ev, subject_names[i], i+1, len(file_dirs)))
        reg = list(region.keys())[0]
        chs = list(region.values())[0]

        # Choose only epochs from the current event
        epochs_ev = epochs[ev]

        ### PRE-EVENT PSD ESTIMATION

        # Calculate Welch's power spectrum density (FFT) for the mean post-event
        [psds_pre, freqs_pre] = spectr.calculate_psd(epochs_ev, subject_names[i], **psd_params['pre'], verbose=False, plot=False)

        # Average all epochs together for each channel and also for each region
        psds_pre = psds_pre.mean(axis=(0))
        df_psds_ch_pre = arrange.array_to_df(subject_names[i], epochs_ev, psds_pre).\
                            reset_index().drop(columns='Subject')

        # Choose only channel of interest data
        # psds_temp_pre = df_psds_ch_pre[ch].to_numpy() # edit if using single channel, maybe only
        psds_temp_pre = df_psds_ch_pre[chs].mean(axis=1).to_numpy()

        ### POST-EVENT PSD ESTIMATION

        # Calculate Welch's power spectrum density (FFT) for the mean post-event
        [psds_post, freqs_post] = spectr.calculate_psd(epochs_ev, subject_names[i], **psd_params['post'], verbose=False, plot=False)

        # Average all epochs together for each channel and also for each region
        psds_post = psds_post.mean(axis=(0))
        df_psds_ch_post = arrange.array_to_df(subject_names[i], epochs_ev, psds_post).\
                            reset_index().drop(columns='Subject')

        # Choose only channel of interest data
        psds_temp_post = df_psds_ch_post[chs].mean(axis=1).to_numpy() # edit if using single channel, maybe only

        ### ERP & POST-minus-ERP PSD ESTIMATIONS

        # Average the event epochs in time domain
        evoked_ev = epochs_ev.average(picks=chs)
        # fig = plt.figure()
        # fig = evoked_ev.plot()
        # plt.show()

        # Calculate Welch's power spectrum density (FFT) for the ERP
        [psds_post_erp, freqs_post] = spectr.calculate_psd(evoked_ev, subject_names[i], **psd_params['post'], verbose=False, plot=False)

        # Calculate the post-minus-ERP PSD subtracting ERP PSD from post-event PSD
        psds_post_diff_ch = psds_temp_post - psds_post_erp.mean(axis=0) # edit if using single channel, maybe only

        ### SPECPARAM

        # Fit the spectrums with FOOOF
        fm_pre = FOOOF(**fooof_params, verbose=True)
        fm_pre.fit(freqs_pre, psds_temp_pre, psd_params['pre']['fminmax'])
        fm_post = FOOOF(**fooof_params, verbose=True)
        fm_post.fit(freqs_post, psds_temp_post, psd_params['post']['fminmax'])
        fm_post_diff = FOOOF(**fooof_params, verbose=True)
        fm_post_diff.fit(freqs_post, psds_post_diff_ch, psd_params['post']['fminmax'])

        def convert_flat_spectr_amplitude(fm, flat_spectr_scale='linear'):
            """Log-linear conversion based on the chosen amplitude scale"""
            if flat_spectr_scale == 'linear':
                flatten_spectrum = 10 ** fm._spectrum_flat
                flat_spectr_ylabel = 'Amplitude (uV\u00b2/Hz)'
            elif flat_spectr_scale == 'log':
                flatten_spectrum = fm._spectrum_flat
                flat_spectr_ylabel = 'Log-normalised amplitude'
            return flatten_spectrum, flat_spectr_ylabel
        
        # Log-linear conversion based on the chosen amplitude scale
        flatten_spectrum_pre, _ = convert_flat_spectr_amplitude(fm_pre, flat_spectr_scale)
        flatten_spectrum_post, _ = convert_flat_spectr_amplitude(fm_post, flat_spectr_scale)
        flatten_spectrum_post_diff, _ = convert_flat_spectr_amplitude(fm_post_diff, flat_spectr_scale)

        # Find individual theta band parameters
        abs_bp_pre, rel_bp_pre = spectr.find_bp(flatten_spectrum_pre, freqs_pre, bands['Theta'])
        abs_bp_post, rel_bp_post = spectr.find_bp(flatten_spectrum_post, freqs_post, bands['Theta'])
        abs_bp_post_diff, rel_bp_post_diff = spectr.find_bp(flatten_spectrum_post_diff, freqs_post, bands['Theta'])

        ### PLOTTING

        def plot_fooof_spectra(fm, flat_spectr_scale, psd_params, bands=None, abs_bp=None, rel_bp=None):
            # Set plot styles
            data_kwargs = {'color' : 'black', 'linewidth' : 1.4, 'label' : 'Original'}
            model_kwargs = {'color' : 'red', 'linewidth' : 1.4, 'alpha' : 0.75, 'label' : 'Full model'}
            aperiodic_kwargs = {'color' : 'blue', 'linewidth' : 1.4, 'alpha' : 0.75,
                                'linestyle' : 'dashed', 'label' : 'Aperiodic model'}
            flat_kwargs = {'color' : 'black', 'linewidth' : 1.4}
            hvline_kwargs = {'color' : 'blue', 'linewidth' : 1.0, 'linestyle' : 'dashed', 'alpha' : 0.75}

            # Log-linear conversion based on the chosen amplitude scale
            flatten_spectrum, flat_spectr_ylabel = convert_flat_spectr_amplitude(fm, flat_spectr_scale)
            
            # Plot power spectrum model + aperiodic fit for MEAN POST-EVENT PSD
            fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), dpi=100)
            plot_spectra(fm.freqs, fm.power_spectrum,
                        ax=axs[0], **data_kwargs)
            plot_spectra(fm.freqs, fm.fooofed_spectrum_,
                        ax=axs[0], **model_kwargs)
            plot_spectra(fm.freqs, fm._ap_fit,
                        ax=axs[0], **aperiodic_kwargs)
            axs[0].set_xlim(psd_params['fminmax'])
            axs[0].grid(linewidth=0.2)
            axs[0].set_xlabel('Frequency (Hz)')
            axs[0].set_ylabel('Log-normalised power (log$_{10}$[µV\u00b2/Hz])')
            axs[0].set_title('Spectrum model fit')
            axs[0].legend()
            
            # Flattened spectrum plot (i.e., minus aperiodic fit)
            plot_spectra(fm.freqs, flatten_spectrum,
                        ax=axs[1], **flat_kwargs)
            if bands!=None:
                axs[1].axvspan(bands[list(bands.keys())[0]][0], bands[list(bands.keys())[0]][1], facecolor='green', alpha=0.2)
            axs[1].set_xlim(psd_params['fminmax'])
            axs[1].grid(linewidth=0.2)
            axs[1].set_xlabel('Frequency (Hz)')
            axs[1].set_ylabel(flat_spectr_ylabel)
            axs[1].set_title('Flattened spectrum')

            # If true, plot all the exported variables on the plots
            if plot_rich == True:
                axs[0].annotate('Error: ' + str(np.round(fm.get_params('error'), 4)) +
                            '\nR\u00b2: ' + str(np.round(fm.get_params('r_squared'), 4)),
                            (0.1, 0.16), xycoords='figure fraction', color='red', fontsize=8.5)
                axs[0].annotate('Exponent: ' + str(np.round(fm.get_params('aperiodic_params','exponent'), 4)) +
                            '\nOffset: ' + str(np.round(fm.get_params('aperiodic_params','offset'), 4)),
                            (0.19, 0.16), xycoords='figure fraction', color='blue', fontsize=8.5)
                if abs_bp!=None and rel_bp!=None:
                    axs[1].annotate('Absolute BP: '+str(np.round(abs_bp, 4))+'\nRelative BP: '+str(np.round(rel_bp, 4)),
                                    (0.69, 0.16), xycoords='figure fraction', color='green', fontsize=8.5)
            
            return fig, axs
        
        fig, axs = plot_fooof_spectra(fm_pre, flat_spectr_scale, psd_params['pre'], 
                                      bands=bands, abs_bp=abs_bp_pre, rel_bp=rel_bp_pre)
        plt.suptitle('Mean pre-event PSD at {} ({})'.format(reg, subject_names[i]))
        plt.tight_layout()
        if savefig == True:
            plt.savefig(fname='{}/{}/FOOOF/{}_{}_{}_{}_mean_pre_event_PSD.png'.format(results_folder, exp_folder,
                                                                        exp_condition, subject_names[i],
                                                                        reg, evname), dpi=300)
        # plt.show()

        fig, axs = plot_fooof_spectra(fm_post, flat_spectr_scale, psd_params['post'], 
                                      bands=bands, abs_bp=abs_bp_post, rel_bp=rel_bp_post)
        plt.suptitle('Mean post-event PSD at {} ({})'.format(reg, subject_names[i]))
        plt.tight_layout()
        if savefig == True:
            plt.savefig(fname='{}/{}/FOOOF/{}_{}_{}_{}_mean_post_event_PSD.png'.format(results_folder, exp_folder,
                                                                        exp_condition, subject_names[i],
                                                                        reg, evname), dpi=300)
        # plt.show()

        fig, axs = plot_fooof_spectra(fm_post_diff, flat_spectr_scale, psd_params['post'], 
                                      bands=bands, abs_bp=abs_bp_post_diff, rel_bp=rel_bp_post_diff)
        plt.suptitle('Post-minus-ERP PSD at {} ({})'.format(reg, subject_names[i]))
        plt.tight_layout()
        if savefig == True:
            plt.savefig(fname='{}/{}/FOOOF/{}_{}_{}_{}_post_minus_erp_PSD.png'.format(results_folder, exp_folder,
                                                                        exp_condition, subject_names[i],
                                                                        reg, evname), dpi=300)
        # plt.show()

        ### EXPORTING

        def ev_to_df(df_ch_ev, i, fm, ch, ev, subject_names, 
                     bands, abs_bp=None, rel_bp=None, stimname='stimulus'):
            if abs_bp == None: abs_bp = np.nan
            if rel_bp == None: rel_bp = np.nan
            df_ch_ev.loc[i, 'Exponent'] = fm.get_params('aperiodic_params','exponent')
            df_ch_ev.loc[i, 'Offset'] = fm.get_params('aperiodic_params','offset')
            df_ch_ev.loc[i, '{} absolute power'.format(list(bands.keys())[0])] = abs_bp
            df_ch_ev.loc[i, '{} relative power'.format(list(bands.keys())[0])] = rel_bp
            df_ch_ev.loc[i, 'R_2'] = fm.get_params('r_squared')
            df_ch_ev.loc[i, 'Error'] = fm.get_params('error')
            df_ch_ev['Channel'] = ch
            df_ch_ev['Event'] = ev
            df_ch_ev['Type'] = stimname
            df_ch_ev['Subject'] = subject_names[i]
            return df_ch_ev
        
        # Add model parameters to dataframe for mean post-event
        df_ch_ev_pre = ev_to_df(df_ch_ev_pre, i, fm_pre, reg, ev, subject_names, 
                                bands, abs_bp_pre, rel_bp_pre, stimname='Mean pre')
        df_ch_ev_post = ev_to_df(df_ch_ev_post, i, fm_post, reg, ev, subject_names, 
                                    bands, abs_bp_post, rel_bp_post, stimname='Mean post')
        df_ch_ev_post_diff = ev_to_df(df_ch_ev_post_diff, i, fm_post_diff, reg, ev, subject_names, 
                                        bands, abs_bp_post_diff, rel_bp_post_diff, stimname='Mean post-ERP')

        # Concatenate to master dataframe for mean post-event
        df_ch = pd.concat([df_ch, df_ch_ev_pre])
        df_ch = pd.concat([df_ch, df_ch_ev_post])
        df_ch = pd.concat([df_ch, df_ch_ev_post_diff])
        

# Reorder the channels and reset index
df_ch = df_ch[['Subject', 'Channel', 'Type', 'Event', 'Exponent', 'Offset',
               '{} absolute power'.format(list(bands.keys())[0]),
               '{} relative power'.format(list(bands.keys())[0]),
               'R_2', 'Error']]
df_ch = df_ch.reset_index(drop=True)

# Export results for post-event data
df_ch.to_excel('{}/{}/FOOOF/{}_{}_specparam.xlsx'.format(results_folder, exp_folder, exp_condition, reg))
display(df_ch)

# Export results with R2 <0.9 to indentify model fits to check
R2_threshold_df = df_ch[df_ch['R_2'] < 0.9]
R2_threshold_df.to_excel('{}/{}/FOOOF/{}_{}_specparam_r2_threshold.xlsx'.format(results_folder, exp_folder, exp_condition, reg))

### ERP DETECTION & IDENTIFICATION

In [ ]:
tminmax = [-200, 800]

# Time windows for different ERP components
erp_wins = {'N1' : [40, 170, -1],
            'N2' : [180, 350, -1],
            'P2' : [100, 260, 1],
            'P3' : [270, 500, 1]}

# Channel of interest: Fz, Cz, Pz
channel_picks = 'Pz' 

# Event names (i.e. different stimuli) within the epochs
# event_list = ['GO trial', 'NO-GO trial']
event_list = {4 : 'GO trial',
              8 : 'NO-GO trial'}

In [ ]:
# Get directories of clean EEG files and set export directory
dir_inprogress = os.path.join(clean_folder, exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress, '_clean-epo.fif')

In [ ]:
# Loop through all the subjects' directories (EEG files directories)
df_erps = pd.DataFrame()
arrange.create_results_folders(exp_folder=exp_folder,results_folder=results_folder, erps=True)
for i in range(len(file_dirs)):
    erp_wins_temp = erp_wins
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]), verbose=False)
    
    # Apply baseline correction
    epochs = epochs.apply_baseline(baseline=(None, 0))

    ### create loop here for going through GO and NO-GO's separately
    for ev, evname in event_list.items():
        print('{} for {} ({}/{})'.format(ev, subject_names[i], i, len(file_dirs)))
        # Create an averaged evoked object from epochs
        evoked_signal = epochs[ev].average(picks=channel_picks)

        # remove or add if save_evoked === truuuu
        evoked_signal.save('{}/{}/ERP analysis/{}_{}_{}_evoked-ave.fif'.format(results_folder, exp_folder,
                                                                            subject_names[i], channel_picks,
                                                                            ev), overwrite=True)

        # Find all the peaks in the evoked signal
        minpeak_times, minpeak_mags, maxpeak_times, maxpeak_mags = erpan.find_all_peaks(evoked_signal, epochs, 
                                                                                        t_range=tminmax, thresh=None,
                                                                                        subject_name=subject_names[i],
                                                                                        verbose=False, plot=False)
        
        # Identify which peaks are which ERPs based on the pre-defined ERP time windows
        erp_peaks, not_erp_peaks = erpan.identify_erps(evoked_signal, erp_wins_temp, minpeak_times, minpeak_mags,
                                                    maxpeak_times, maxpeak_mags, t_range=tminmax, subject_name=subject_names[i],
                                                    verbose=False, plot=True, savefig=False,
                                                    results_foldername=results_folder, exp_folder=exp_folder)

        # After visual inspection, it's possible to re-define the time windows to look for the peak
        while input('Do you need to do any manual time window changes? (leave empty if "no")') != '':
            print('Changing time window parameters for {}'.format(subject_names[i]))
            new_time_win = [None, None, None]

            # Ask user for which ERP they want to change or add
            erp_tochange = input('What ERP time window you want to change (e.g., N1)?')

            # Ask user what should be the minimum timepoint of the time window for that ERP
            new_time_win[0] = int(input('Enter MIN time of the window in interest for {} (e.g., 50)'.format(erp_tochange)))

            # Ask user what should be the maximum timepoint of the time window for that ERP
            new_time_win[1] = int(input('Enter MAX time of the window in interest for {} (e.g., 100)'.format(erp_tochange)))

            # Ask user whether this ERP should be a postitive (1) or negative (-1) peak
            new_time_win[2] = int(input('Enter whether to look for MIN (-1) or MAX (1) voltage for {}'.format(erp_tochange)))

            # Change the temporary ERP time window parameters to the user inputted parameters
            erp_wins_temp[erp_tochange] = new_time_win
            print('Changing', erp_tochange, 'with new time window:', str(new_time_win))

            # Use these new parameters to find either minimum or maximum value in that range
            try:
                erp_peaks = erpan.find_minmax_erp(evoked_signal, erp_peaks, erp_tochange, new_time_win,
                                                t_range=tminmax, subject_name=subject_names[i], verbose=False, plot=True,
                                                savefig=False, results_foldername=results_folder, exp_folder=exp_folder)
            except:
                print('Something went wrong with manual ERP detection, try again.')

        # Add this/these new temporary ERP to the main dataframe
        df_erps_temp = erpan.erp_dict_to_df(erp_peaks, erp_wins_temp, subject_names[i])
        df_erps_temp['Event'] = ev
        df_erps_temp['Channel'] = channel_picks
        df_erps = pd.concat([df_erps, df_erps_temp])
        print('ERPs have been found and added to the dataframe for {}'.format(subject_names[i]))
        display(df_erps)

# Calculate relative peak-to-peak amplitudes between the ERPs
print('Adding relative amplitudes for N1-P2, P2-N2, N2-P3')
df_erps['N1-P2 amplitude'] = df_erps['P2 amplitude'] - df_erps['N1 amplitude']
df_erps['P2-N2 amplitude'] = df_erps['N2 amplitude'] - df_erps['P2 amplitude']
df_erps['N2-P3 amplitude'] = df_erps['P3 amplitude'] - df_erps['N2 amplitude']

# Export all the detected ERPs to an Excel spreadsheet
display(df_erps)
df_erps.to_excel('{}/{}/ERP analysis/{}_{}_grandaverage_erps.xlsx'.format(results_folder,exp_folder,exp_condition,channel_picks))

### DATA VISUALISATION: ERPs

In [ ]:
# Export the figure to results folder or not
savefig = True

# Subjects which to not plot
exclude_subjects = [] # ['OKTOS_0019', 'OKTOS_0024', 'OKTOS_0033']

# Channel of interest
ch = 'Pz'

# Event names (i.e. different stimuli) within the epochs
event_list = ['GO trial', 'NO-GO trial']

In [ ]:
sns.set_theme(context='notebook', font_scale=1.3,
              style='whitegrid', palette='muted',
              font='sans-serif')

# Get directories of clean EEG files and exclude the pre-defined subjects
dir_inprogress = os.path.join(clean_folder, exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress, "_clean-epo.fif",
                                      exclude_subjects=exclude_subjects)

# Loop through all the subjects' directories (EEG files directories)
evoked_signal_go = [None]*len(file_dirs)
evoked_signal_nogo = [None]*len(file_dirs)
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress,
                                                                subject_names[i]),
                                                                verbose=False)
    
    # Create an averaged evoked object from epochs for both events
    evoked_signal_go[i] = epochs['GO trial'].average(picks=ch)
    evoked_signal_nogo[i] = epochs['NO-GO trial'].average(picks=ch)

# Average all the averaged evoked objects, thereby creating a grand average signals
go_master_grand_evoked_data = mne.grand_average(evoked_signal_go).data[0]*1e6
go_master_grand_evoked_times = mne.grand_average(evoked_signal_go).times*1e3
nogo_master_grand_evoked_data = mne.grand_average(evoked_signal_nogo).data[0]*1e6
nogo_master_grand_evoked_times = mne.grand_average(evoked_signal_nogo).times*1e3

# Plot all experiments' grand average signals on a single plot
fig, ax = plt.subplots(figsize=(6, 4), layout='tight', dpi=150)
ax.plot(go_master_grand_evoked_times, go_master_grand_evoked_data, linewidth=3)
ax.plot(nogo_master_grand_evoked_times, nogo_master_grand_evoked_data, linewidth=3)
ax.legend(event_list)
ax.set_title('Grand average of all participants at {}'.format(ch))
ax.set_xlim([-200, 1000])
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (µV)')
ax.grid(which='major', axis='y', alpha=0.2)
ax.grid(which='major', axis='x', alpha=0.7)
if savefig == True:
    plt.savefig(fname='{}/{}/GRAND_erpfig_{}.png'.format(results_folder, exp_folder, ch),
                dpi=300)
plt.show()

In [ ]:
df_ch

In [ ]:
df_pre_go = df_ch.loc[(df_ch['Type']=='Mean pre') & (df_ch['Event']=='4'), :]
df_post_erp_go = df_ch.loc[(df_ch['Type']=='Mean post-ERP') & (df_ch['Event']=='4'), :]

In [ ]:
df_post_erp_go

In [ ]:
df_post_erp_go['Exponent'].to_numpy()

In [ ]:
df_pre_go['Exponent']

In [ ]:
df_post_erp_go['Exponent'] - df_pre_go['Exponent']

In [ ]:
df_ch[df_ch['Type']=='Mean pre']